# Social Computing - Summer 2018
# Exercise 3: Collaborative Filtering Recommender System

Write a simple collaborative filtering movie recommender in python. The input (attached to this exercise) is a subset of the MovieLens dataset (movielens.org), which contains 1862 movies, and 100K ratings. Find the dataset Piazza. The input is provided in two data files: u.item: is a listing of movies. Each row represents a movie with its attributes separated by ‘|’. We are only interested in the first two attributes which are the movie ID and the movie name. The second data file is u.data, which contains the movie ratings by users. A single row represents one user’s rating for one movie, and the attributes are (from left to right): user ID, movie ID, rating, and timestamp. 

<b>Exercise</b><br>
The entry point to your recommendation engine should be a python method called “recommend” that takes a user ID, and paths to the two aforementioned data files. The method should return the top twenty recommended movies for that user <u><b>(The movies should not have been rated by that user before)</b></u>. The output should be a list of python tuples (sorted by recommended movies' expected ratings: highest first). Each tuple has the following two attributes: movie name, expected rating. You are free to design your recommendation engine the way you want but straightforward collborative filtering is highly recommended. Make sure that the code is clean, readable, and well documented. Test the code for user 15!

### The Collaborative Filtering Recommender
The idea of collaborative filtering is to find similar users to the target user. The items highly rated by those users are likely to be favorited by the target user. Therefore, the main problem is to find the list of users similar to the target user. There are several ways of measuring similarity. We could e.g. use cosine similarity but one of the simplest measures is Euclidean distance. Your program should calculate the absolute Euclidean distance between the target user and all other users in the dataset and calculate the expected rating for the target user for each movie in the dataset based on the forumla:

$$r_{ui} = \frac{\sum_{v \in N_i(u)} w_{uv}r_{vi}}{\sum_{v \in N_i(u)} w_{uv}}$$

Where $r_{ui}$ is the expected recommendation of item i for target user u. $N_i(u)$ is the set of similar users to target user u for the designated item i. $w_{uv}$ is the similarity score between users u and v (used as a weighting factor) for $r_{vi}$ which is the rating of user v for item i. 
The curious student should refer to: Ricci et al (eds.) "Recommender Systems Handbook", Springer 2011, for more details

In [28]:
# Method has following parameters:
# path_u_item: path to u.item as a string
# HINT: movie_dictionary = {movie_id: movie_name}
# TODO Parse u.item
def create_movie_dictionary(path_input):
    movie_dictionary = {}
    for line in open(path_input):
        movie_data = line.strip().split('|')
        movie_dictionary[int(movie_data[0])] = movie_data[1]  
    return movie_dictionary


# Create a dictionary: {user_id: {movie_id: user_rating}}
def create_user_rating_dictionary(path_input):
    user_rating_dictionary = {}
    for line in open(path_input):
        row = line.strip().split('\t')
        user_id, movie_id, rating, timestamp = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        try:
            user_rating_dictionary.setdefault(user_id, {})
            user_rating_dictionary[user_id][movie_id] = rating
        except KeyError:
            print "key error found! " + user_id + " " + movie_id
    return user_rating_dictionary


# Using Euclidean distance to calculate similarity score
def calculate_similarity_score(ratings, user_id1, user_id2):
    common_movies = []
    # TODO Find common movies rated by both users
    user1_movies = ratings[user_id1]    
    user2_movies = ratings[user_id2]
    
    for movie in user2_movies:
        if movie in user1_movies:
            common_movies.append(movie)            
            
    if len(common_movies) == 0: # no common ratings between two users. Similarity is 0
        return 0

    # TODO Calculate Euclidean distance between two users based on their common ratings
    sum_of_squares_of_differences = 0
    for movie_id in common_movies:    
        sum_of_squares_of_differences += (user1_movies[movie_id] - user2_movies[movie_id])**2
        
        # TODO Accumulate the sum of squares of differences in ratings between the two users for the same movie
    return 1 / (1 + sum_of_squares_of_differences)   

    

def cf_recommend(ratings, target_user_id):
    weighted_ratings = {} # {movie_id: weighted_rating}
    similarity_scores = {} # {movie_id: similarity_score}
    recommended_movie_list = [] # Each element is a tuple (estimated_rating, movie_id)
    for user_id in ratings:
        if user_id != target_user_id:
            similarity_score = calculate_similarity_score(ratings, target_user_id, user_id)
            if similarity_score > 0:
                for movie_id in ratings[user_id]:
                    # Movie was not recommended by the target user before
                    if movie_id not in ratings[target_user_id]:
                        if movie_id not in weighted_ratings:
                            weighted_ratings[movie_id] = ratings[user_id][movie_id]*similarity_score
                        else:
                            weighted_ratings[movie_id] += ratings[user_id][movie_id]*similarity_score
                        if movie_id not in similarity_scores:
                            similarity_scores[movie_id] = similarity_score
                        else:
                            similarity_scores[movie_id] += similarity_score
                        # TODO Accumulate the weighted rating for that movie
                        # The weighted rating of the movie = user_id's rating of that movie * similarity 
                        # score between that user and the target_user 
                        # of that user to the target_user
                        # TODO Accumulate the similarity scores of all users who rated that movie

    for movie_id in weighted_ratings:
        estimated_rating = weighted_ratings[movie_id] / similarity_scores[movie_id]
        # TODO Weighted_rating/sigma (similarity scores of users who rated that movie)
        recommended_movie_list.append((estimated_rating, movie_id))
    
    # TODO Sort the list
    recommended_movie_list.sort(key=lambda sort_attr: sort_attr[0], reverse=True)
    return recommended_movie_list
# List of recommended movies for user_id from highest to lowest estimated rating
    

# Testing implementation

movie_dict = create_movie_dictionary("u.item")
ratings = create_user_rating_dictionary("u.data")
recommended_movies = cf_recommend(ratings,15)
top_twenty = []


for estimated_rating, movie_id in recommended_movies[:20]:    
    top_twenty.append((movie_dict[movie_id], estimated_rating))
print top_twenty


[('Fire Down Below (1997)', 5), ('Paths of Glory (1957)', 5), ('Empire Strikes Back, The (1980)', 5), ('Princess Bride, The (1987)', 5), ('Alien (1979)', 5), ('Godfather: Part II, The (1974)', 5), ('Graduate, The (1967)', 5), ('Hoodlum (1997)', 5), ('187 (1997)', 5), ('Soul Food (1997)', 5), ('Money Talks (1997)', 5), ("Eve's Bayou (1997)", 5), ('North by Northwest (1959)', 5), ('Killing Fields, The (1984)', 4), ('Shine (1996)', 4), ('Hoop Dreams (1994)', 4), ('Apocalypse Now (1979)', 4), ('Quiz Show (1994)', 4), ('Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)', 4), ('Piano, The (1993)', 4)]


In [ ]:
# TODO: Least misery strategy for group recommendations aggregation
import math
def least_misery(social_preds, restaurants, group):   
    group_ratings = []
    # TODO: implement strategy
    for restaurant in restaurants:
        agg_rating = 0
        for member in social_preds:
            agg_rating += social_preds[member][restaurant]
        group_ratings.append((restaurant, agg_rating))
    group_ratings.sort(key = lambda tup: tup[1], reverse = True) 
    return group_ratings
    return group_rating_dict

# Using Euclidean distance to calculate similarity score
def calculate_similarity_score(ratings, user_id1, user_id2):
    
    common_movies = []
    # TODO Find common movies rated by both usersu
    movies1 = ratings[user_id1].keys()
    movies2 = ratings[user_id2].keys()
    common_movies = list(movies1 & movies2)
            
    if len(common_movies) == 0: # no common ratings between two users. Similarity is 0
        return 0

    # TODO Calculate Euclidean distance between two users based on their common ratings
    sum_of_squares_of_differences = 0
    for movie_id in common_movies:
        # TODO Accumulate the sum of squares of differences in ratings between the two users for the same movie
        sum_of_squares_of_differences += (ratings[user_id1][movie_id] - ratings[user_id2][movie_id])**2
        sum_of_squares_of_differences = math.sqrt(sum_of_squares_of_differences)
        
    return 1 / (1 + sum_of_squares_of_differences)
    

def cf_recommend(target_user_id, restaurants, ratings, avg_ratings):
    weighted_ratings = {} # {movie_id: weighted_rating}
    similarity_scores = {} # {movie_id: similarity_score}
    recommended_movie_list = {} # Each element is a tuple (estimated_rating, movie_id)
    for user_id in ratings:
        if user_id != target_user_id:
            similarity_score = calculate_similarity_score(ratings, target_user_id, user_id)
            if similarity_score > 0:
                for movie_id in ratings[user_id]:
                    # Movie was not recommended by the target user before
                    if movie_id not in ratings[target_user_id]:
                        try:
                            if movie_id in weighted_ratings:
                                weighted_ratings[movie_id] += ratings[user_id][movie_id]*similarity_score
                                similarity_scores[movie_id] += similarity_score
                            else:
                                weighted_ratings[movie_id] = ratings[user_id][movie_id]*similarity_score
                                similarity_scores[movie_id] = similarity_score
                        except KeyError:
                            print("key error found! " + str(user_id) + " " + str(movie_id))
  
    for key in weighted_ratings: # TODO for each movie
        # TODO Weighted_rating/sigma (similarity scores of users who rated that movie)
        estimated_rating = weighted_ratings[key]/similarity_scores[key]
        recommended_movie_list[key] = estimated_rating
       
    for restaurant in restaurants:
        if restaurant not in weighted_ratings:
            recommended_movie_list[restaurant] = avg_ratings[restaurant]
        
        
    return recommended_movie_list # List of recommended movies for user_id from highest to lowest estimated rating


def calculate_base_predictions(participant, restaurants, ratings,average_rating):
    #TODO: Reuse your collaborative filtering recommender from exercise 3
    return cf_recommend(participant, restaurants, ratings, average_rating)

def calculate_social_context_aware_predictions(group, restaurants, base_predicted_ratings,dom_expertise):
    #TODO: Apply the formula for the participant for each restaurant
    social_preds = {}
    for member in group:
        for restaurant in restaurants:
            if member not in social_preds:
                social_preds[member] = {}
            dom_sum = 0
            dom_base_prod = []
            for other_members in group:
                if member != other_members:
                    dom_sum += dom_expertise[str(member)][str(other_members)]
                    dom_base_prod.append(dom_expertise[str(member)][str(other_members)]*base_predicted_ratings[str(other_members)][str(restaurant)])

            social_preds[member][restaurant] = (1/dom_sum)*(sum(dom_base_prod))    
            
    return social_preds # {participant_id: {restaurant_id: social_rating}}

def get_restaurants(ratings_path): 
    #TODO: using the ratings return a list of unique restaurant IDs
    restaurants_list = list()
    f = open(ratings_path, 'r')
    lines = f.readlines()[1:]
    for line in lines:    
        rating_data = line.strip().split(',')
        if rating_data[1] not in restaurants_list:
            restaurants_list.append(rating_data[1])
    
    return restaurants_list


def get_participants(ratings_path): 
    #TODO: using the ratings return a list of unique restaurant IDs
    participants = []
    f = open(ratings_path, 'r')
    lines = f.readlines()[1:]
    
    for line in lines:
        ratings_data = line.split(',')
        participants.append(ratings_data[0])        
    participants = set(participants)
    return participants

    
def create_rating_dictionary(ratings_path):
    ratings = {}
    average_rating = {}
    num_ratings_per_rest = {}
    f = open(ratings_path, 'r')
    lines = f.readlines()[1:]
    for line in lines:    
        data = line.strip().split(',')
        if data[0] not in ratings:       
            ratings[int(data[0])] = {}
        ratings[int(data[0])][data[1]] =  ((float(data[2])/100)+(float(data[3])/100)+(float(data[4])/100)+(float(data[5])/100)+(float(data[6])/100)+(float(data[7])/100))/6        
    
        if data[1] not in average_rating:
            average_rating[data[1]] = []
        average_rating[data[1]].append(((float(data[2])/100)+(float(data[3])/100)+(float(data[4])/100)+(float(data[5])/100)+ (float(data[6])/100)+(float(data[7])/100))/6)
        
    for restaurant in average_rating:        
        average_rating[restaurant] = sum(average_rating[restaurant])/len(average_rating[restaurant])
    
    return ratings,average_rating


def domain_expertise_parser(domain_expertise_path):
    domain_dict = {}
    f = open(domain_expertise_path, 'r')
    lines = f.readlines()[1:]
    for line in lines:
        domain_data = line.split(',')
        if domain_data[0] not in domain_dict:
            domain_dict[domain_data[0]] = {}
        domain_dict[domain_data[0]][domain_data[1]] = float(domain_data[2])/100        
    
    return domain_dict

# Group recommender (Main program)
def group_recommender(group, ratings_path, domain_expertise_path):
    # Pre-processing:
    #-----------------
    # TODO: parse ratings.csv (e.g into a dictionary of ratings)
    # parse domain_expertise.csv (e.g into the dictionary {from: {to, domain_expertise}})
    # restaurants = get_restaurants(ratings) --> from the ratings dictionary, extract the list of restaurant IDs (a list of unique restaurant IDs)
    
    ratings,average_rating = create_rating_dictionary(ratings_path)
    restaurants = get_restaurants(ratings_path)
    domain_dict = domain_expertise_parser(domain_expertise_path)
    participants = list(get_participants(ratings_path))
    # Calculate single user recommendations (predicted ratings):
    #-------------------------------------------------------------
    # For each participant in the group, calculate the base ratings for each restaurant
    # For each participant in the group, calculate the social-context-aware predicted ratings (given the base predicted ratings)
    base_preds = {}
    for participant in participants:
        base_preds[participant] = calculate_base_predictions(participant, restaurants, ratings, average_rating)
    # Calculate group recommendations (group predicted ratings) - based on the "Least Misery" strategy:
    #--------------------------------------------------------------------------------------------------
    # Given the social-context-aware predicted ratings for each group member, aggregate those ratings 
    # into group recommendations for each restaurant based on the "Least Misery" strategy (sorted by predicted ratings: highest first)
    social_preds = {}
    social_preds = calculate_social_context_aware_predictions(group, restaurants, base_preds, domain_dict)
    group_ratings = least_misery(social_preds, restaurants, group)-
    
    for i in range(len(group_ratings)):
        print(group_ratings[i])


# Test (Call your main program to test it with the sample groups from the exercise description above)
group_recommender([160, 161, 162], "ratings.csv", "domain_expertise.csv")